In [ ]:
# default_exp web

# Web
>This module contain functions to process the model outputs for web applications.

In [ ]:
# hide
from nbdev.showdoc import show_doc
from nbdev.export import notebook2script

In [ ]:
# export
from pathlib import Path
import scipy.io as sio
import pandas as pd
import geopandas as gp
import numpy as np
from tqdm import tqdm
import rasterio
import rasterio.features
import pdb
import shapely
from IPython.core.debugger import set_trace
from shapely.geometry import Polygon
from banet.core import *
from banet.geo import  *
from banet.predict import split_mask
Path.ls = ls

In [ ]:
# export
def ba_split(io_path:InOutPath, region:Region, threshold=0.5, min_size=1, keys=['burned', 'date']):
    "Computes burned areas shapefiles."
    files = sorted(io_path.src.ls())
    print(f'Processing {files[-1]}')
    data = sio.loadmat(files[-1])
    fires = split_mask((data[keys[0]]>=threshold).astype(np.uint8), thr_obj=min_size)
    
    times = pd.DatetimeIndex([pd.Timestamp(o) for o in data['times']])
    doy = times.dayofyear.values.astype(int)

    data[keys[1]][np.isnan(data[keys[1]])] = 0
    data[keys[1]] = doy[data[keys[1]].astype(int)]
    data[keys[1]] = data[keys[1]].astype(np.float16)
    data[keys[1]][data[keys[1]]==doy[0]] = np.nan

    gpd = gp.GeoDataFrame()
    for fire in tqdm(fires):
        dates = data[keys[1]].copy()
        dates[fire==0] = np.nan
        start_date = np.nanmin(dates)
        end_date = np.nanmax(dates)
        geoms = list({'properties': {'raster_val': v}, 'geometry': s}
            for i, (s, v) in enumerate(rasterio.features.shapes(fire.astype(np.uint8), transform=region.transform))
        )
        g0 = gp.GeoDataFrame.from_features(geoms, crs={'init' :'epsg:4326'})
        g0['start'] = start_date.astype(int)
        g0['end'] = end_date.astype(int)
        gpd = pd.concat((gpd, g0), axis=0)

    gpd = gpd.loc[gpd.raster_val==1.0].reset_index(drop=True)
    gpd = gp.GeoDataFrame(gpd, crs={'init' :'epsg:4326'})
    gpd['area'] = (gpd.to_crs(epsg=25830).area.values*0.0001).astype(int) # TODO: EPGS for other regions
    gpd = gpd.drop('raster_val', axis=1)
    return gpd, data, fires

def fires2raster(path_save, fires, data, region:Region, keys=['burned', 'date']):
    "Saves raster files for individual fires."
    for i, f in enumerate(fires):
        args = np.argwhere(f>0)
        lon, lat = region.coords()
        (rmin, cmin), (rmax, cmax) = args.min(0), args.max(0)
        rmax += 1
        cmax += 1
        lat_r = lat[rmin-1:rmax+1]
        lon_r = lon[cmin-1:cmax+1]
        tfm = rasterio.Affine(region.pixel_size, 0, lon_r.min(), 0, -region.pixel_size, lat_r.max())
        burned_r = data[keys[0]][rmin-1:rmax+1, cmin-1:cmax+1].copy().astype(np.float16)
        date_r =  data[keys[1]][rmin-1:rmax+1, cmin-1:cmax+1].copy().astype(np.float16)
        burned_r[f[rmin-1:rmax+1, cmin-1:cmax+1]==0] = np.nan
        date_r[f[rmin-1:rmax+1, cmin-1:cmax+1]==0] = np.nan

        burned_r = burned_r*255
        burned_r[np.isnan(burned_r)] = 0
        burned_r = burned_r.astype(np.uint16)
        date_r[np.isnan(date_r)] = 0
        date_r = date_r.astype(np.uint16)
        raster = np.array((burned_r, date_r))

        meta = {
            'driver': 'GTiff', 
            'dtype': 'uint16', 
            'nodata': None, 
            'width': burned_r.shape[1], 
            'height': burned_r.shape[0], 
            'count': 2, 
            'crs': rasterio.crs.CRS.from_epsg(4326), 
            'transform': tfm
        }

        with rasterio.open(path_save/f'ba_{i}.tif', 'w', **meta) as dst:
            dst.write(raster)
    
def out2raster(iop:InOutPath, region:Region, keys=['burned', 'date']):
    "Saves raster files for last output."
    file = sorted(iop.src.ls())[-1]
    print(f'Processing {file}')
    data = sio.loadmat(file)
    
    lon, lat = region.coords()
    burned_r = data[keys[0]]
    date_r =  data[keys[1]]
    burned_r = burned_r*255
    burned_r[np.isnan(burned_r)] = 0
    burned_r = burned_r.astype(np.uint16)
    date_r[np.isnan(date_r)] = 0
    date_r = date_r.astype(np.uint16)
    raster = np.array((burned_r, date_r))

    meta = {
        'driver': 'GTiff', 
        'dtype': 'uint16', 
        'nodata': None, 
        'width': burned_r.shape[1], 
        'height': burned_r.shape[0], 
        'count': 2, 
        'crs': rasterio.crs.CRS.from_epsg(4326), 
        'transform': region.transform
    }

    with rasterio.open(iop.dst/f'{file.stem}.tif', 'w', **meta) as dst:
        dst.write(raster)
    
def process_last(iop:InOutPath, region:Region, filter_region:Polygon, threshold=0.5, 
                 min_size=1, keys=['burned', 'date']):
    df, data, fires = ba_split(iop, region, min_size=min_size, threshold=threshold, keys=keys)
    df = df.loc[df.within(filter_region)]
    fires = np.array(fires, dtype=np.uint16)
    fires = fires[df.index.values.reshape(-1)]
    df = df.reset_index(drop=True)
    fires2raster(iop.dst, fires, data, region, keys=keys)
    df.to_file(iop.dst/'banet_nrt.shp')

Example: 
```python
iop = InOutPath('outputs', 'web')
R = Region.load('config/R_PT.json')
filter_region = Polygon([[-10, 44], [-6, 44], [-6, 36], [-10, 36]])
process_last(iop, R, filter_region)
```

In [ ]:
# hide
notebook2script()

Converted 00_core.ipynb.
Converted 01_geo.ipynb.
Converted 02_data.ipynb.
Converted 03_models.ipynb.
Converted 04_predict.ipynb.
Converted 04b_nrt.ipynb.
Converted 04c_historical.ipynb.
Converted 05_train.ipynb.
Converted 06_cli.ipynb.
Converted 07_web.ipynb.
Converted index.ipynb.
Converted tutorial.australia2020.ipynb.
Converted tutorial.australia2020_100m.ipynb.
